<a href="https://colab.research.google.com/github/kookeej/DILAB/blob/main/7.1-7.7/lab07-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lab07-1 - Tips
===
여러가지 tip들에 대해 다뤄보는 시간이다.

## 목차
* Maximum Likelihood Estimation
* Overfitting and Regularization
* Training and Test Dataset
* Learning Rate
* Data Preprocessing

# 1. Maximum Likelihood Estimation (MLE)
예를 들어 압정 앞/뒷면이 나올 확률을 살펴보자. 앞면, 그리고 뒷면 두 가지 경우만 존재하므로 이항분포, 또는 베르누이 분포라고도 한다. 따라서 ```Binary classification```을 수행하면 된다.     
MLE는 말 그대로 likelihood가 최대가 되는 지점을 찾는 것이다. 만약 최대가 되는 theta값이 0.27이라면, Observation을 가장 잘 설명하는 theta는 0.27이다.     
즉, Maximum likelihood extimation은 observation을 가장 잘 설명하는 theta를 찾는 과정이다.    
관찰한 data를 가장 잘 설명하는 어떤 확률분포함수의 파라미터를 찾아내는 과정이다.    

* Gradient descent를 통해 최적화한다.
* 하지만 MLE의 경우 overfitting을 피할 수 없다.

# 2. Overfitting
* MLE를 통해 model을 얻으면 과하게 구불거리는 곡선이 나온다. 이를 ```overfitting```이라고 한다. 우리는 이 overfitting을 최소화해야한다. overfitting을 최소화하는 방법에는 Training set과 Test set을 나누는 방법이 있다.


# 3. Training and Test Dataset


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
torch.manual_seed(1)

* Training set과 Test set을 나눠준다.

In [3]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4,],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

In [4]:
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

* |x_train| = (m, 3)
* |y_train| = (m,)
* |x_test| = (m`, 3)
* [y_test| = (m`, )

* Model을 설계해준다.

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)       # 3->3
    def forward(self, x):        
        return self.linear(x)               # |x| = (m, 3) -> (m, 3)

In [6]:
model = SoftmaxClassifierModel()

In [7]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

* Training

In [8]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

* Test

In [9]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)

    print('Accuracy: {}% Cost: {:.6f}'.format(
        correct_count / len(y_test) * 100, cost.item()
    ))

* Result

In [10]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


MLE를 통해서 가장 적절한 ```theta```값을 찾는 과정이다.

In [11]:
test(model, optimizer, x_test, y_test)

Accuracy: 0.0% Cost: 1.425844


하지만 test set에 대해서 똑같이 수행해보았을 때, cost값이 높아진 것을 확인할 수 있다. 따라서 어떤 epoch 시점에서 pick을 해봤을 때는 이미 overfitting이 발생한 상태이다.

# 4. Learning Rate
```learning rate```는 학습 속도를 조절한다.
## 4.1. Overshooting
```learning rate```가 너무 크면 ```overshooting```이 발생한다.

In [12]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e5)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 976950.750000
Epoch    2/20 Cost: 1279135.250000
Epoch    3/20 Cost: 1198378.875000
Epoch    4/20 Cost: 1098825.750000
Epoch    5/20 Cost: 1968197.750000
Epoch    6/20 Cost: 284763.250000
Epoch    7/20 Cost: 1532260.250000
Epoch    8/20 Cost: 1651503.750000
Epoch    9/20 Cost: 521878.593750
Epoch   10/20 Cost: 1397263.250000
Epoch   11/20 Cost: 750986.375000
Epoch   12/20 Cost: 918691.375000
Epoch   13/20 Cost: 1487888.250000
Epoch   14/20 Cost: 1582260.250000
Epoch   15/20 Cost: 685818.125000
Epoch   16/20 Cost: 1140048.875000
Epoch   17/20 Cost: 940566.375000
Epoch   18/20 Cost: 931638.250000
Epoch   19/20 Cost: 1971322.750000


```learning rate```가 너무 크기 때문에 cost값이 발산해버리는 문제가 발생했다.

## 4.2. Too small 
```learning rate```가 너무 작으면 학습 속도가 너무 느려지기 때문에 cost가 거의 줄어들지 않는다.

In [13]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-10)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 3.187324
Epoch    1/20 Cost: 3.187324
Epoch    2/20 Cost: 3.187324
Epoch    3/20 Cost: 3.187324
Epoch    4/20 Cost: 3.187324
Epoch    5/20 Cost: 3.187324
Epoch    6/20 Cost: 3.187324
Epoch    7/20 Cost: 3.187324
Epoch    8/20 Cost: 3.187324
Epoch    9/20 Cost: 3.187324
Epoch   10/20 Cost: 3.187324
Epoch   11/20 Cost: 3.187324
Epoch   12/20 Cost: 3.187324
Epoch   13/20 Cost: 3.187324
Epoch   14/20 Cost: 3.187324
Epoch   15/20 Cost: 3.187324
Epoch   16/20 Cost: 3.187324
Epoch   17/20 Cost: 3.187324
Epoch   18/20 Cost: 3.187324
Epoch   19/20 Cost: 3.187324


cost값이 거의 줄어들지 않는 것을 확인할 수 있다. 따라서 적절한 learning rate을 설정하는 것이 중요하다는 것을 알 수 있다.

In [14]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.341574
Epoch    1/20 Cost: 1.198802
Epoch    2/20 Cost: 1.150877
Epoch    3/20 Cost: 1.131977
Epoch    4/20 Cost: 1.116242
Epoch    5/20 Cost: 1.102514
Epoch    6/20 Cost: 1.089676
Epoch    7/20 Cost: 1.077479
Epoch    8/20 Cost: 1.065775
Epoch    9/20 Cost: 1.054511
Epoch   10/20 Cost: 1.043655
Epoch   11/20 Cost: 1.033187
Epoch   12/20 Cost: 1.023091
Epoch   13/20 Cost: 1.013356
Epoch   14/20 Cost: 1.003968
Epoch   15/20 Cost: 0.994917
Epoch   16/20 Cost: 0.986189
Epoch   17/20 Cost: 0.977775
Epoch   18/20 Cost: 0.969661
Epoch   19/20 Cost: 0.961836


적절한 learning rate을 설정했을 때 cost값이 제대로 줄어드는 것을 알 수 있다. 보통 0.01로 초기화한다.

# 5. Data Preprocessing
데이터 전처리를 통해 데이터를 좀 더 학습하기 쉽도록 바꿔주는 것은 매우 중요하다.

In [15]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

데이터 전처리를 해주게 되면 훨씬 더 학습이 수월해진다.     
데이터를 zero-center하고 normalize한다.    

image.png
     

여기서 σ는 standard deviation, μ는 평균값이다.


In [16]:
mu = x_train.mean(dim=0)
sigma = x_train.std(dim=0)
norm_x_train = (x_train - mu) / sigma
print(norm_x_train)

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])


즉, 단위를 통일하는 표준화를 한다???
training set이 정규분포가 된다???? (질문)

* Training with Preprocessed Data

In [17]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
    
    def forward(self, x):
        return self.linear(x)

In [18]:
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [19]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        prediction = model(x_train)

        cost = F.mse_loss(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [20]:
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29785.089844
Epoch    1/20 Cost: 18906.166016
Epoch    2/20 Cost: 12054.673828
Epoch    3/20 Cost: 7702.029785
Epoch    4/20 Cost: 4925.733398
Epoch    5/20 Cost: 3151.632812
Epoch    6/20 Cost: 2016.996094
Epoch    7/20 Cost: 1291.051270
Epoch    8/20 Cost: 826.505249
Epoch    9/20 Cost: 529.207397
Epoch   10/20 Cost: 338.934174
Epoch   11/20 Cost: 217.153564
Epoch   12/20 Cost: 139.206757
Epoch   13/20 Cost: 89.313782
Epoch   14/20 Cost: 57.375465
Epoch   15/20 Cost: 36.928429
Epoch   16/20 Cost: 23.835773
Epoch   17/20 Cost: 15.450401
Epoch   18/20 Cost: 10.077809
Epoch   19/20 Cost: 6.633700


cost값이 줄어드는 것을 확인할 수 있다. 만약 우리가 데이터 전처리를 수행하지 않았다면 데이터 최적화에 더 어려움을 겪었을 수도 있다. 특히 ```y_train```의 크기가 커지거나 원소의 값의 차이가 매우 크게 날 때 전처리가 필요하다.